In [1]:
# -*- coding: utf-8 -*-
import scipy.io as scio
#import pandas as pd
import numpy as np
import random
import operator
import copy
import os
import math
#import matplotlib.pyplot as plt
#import matlab.engine
import time
import multiprocessing as mp
import warnings

In [2]:
# loading instance data
def load_data(file_path):
    
    matrix_data = scio.loadmat(file_path)
    google_distance = matrix_data['GoogleDistances']
    great_distance = matrix_data['GreatCircleDistances']
    XY = matrix_data['MyLocations']
    num_node = len(google_distance)
    location = np.ones((num_node,num_node),dtype=float)
    
    for i in range(num_node):
        google_distance[i][i] = 0
        great_distance[i][i] = 0 
        
    for i in range(num_node):
        location[i,0] = XY[i][0][0,0]
        location[i,1] = XY[i][0][0,1]
    
    return google_distance, great_distance, location

In [3]:
def element_inquiry(greatcir_dis,max_node):
    
    n_node = len(greatcir_dis)
    logical_matrix = np.zeros((n_node,n_node),dtype=int)
    
    logical_matrix[0] = 1
    logical_matrix[:,0] = 1
    
    # according to the inquiry budget, specify the elements which are available from google distance matrix
    
    for k in range(n_node):
        for row_id in range(n_node):
            cur_row_data = greatcir_dis[row_id]
            cur_row_data[row_id] = 100000000000
            near_node_id = np.argsort(cur_row_data)[k]
            cur_row_data[row_id] = 0
            logical_matrix[row_id,near_node_id] = 1
            logical_matrix[near_node_id,row_id] = 1
            if sum(sum(logical_matrix)) >= max_node:
                return logical_matrix
        
    

In [4]:
def gcdist(coordinates1,coordinates2):
    
    lat1 = coordinates1[0]
    lon1 = coordinates1[1]
    lat2 = coordinates2[0]
    lon2 = coordinates2[1]
    radius = 6371000  # m

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a1 = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a1), math.sqrt(1 - a1))
    d = radius * c
    return d
    

In [125]:
def knn_matrix_completion(logi_mat,greatcir_dis,circuity_factor,num_adj,sf,location,query_file_id):
    
    query_file_name = "query_data_" + query_file_id + ".txt"
    print(query_file_name)
    
    history_data = np.loadtxt(query_file_name)
    print("the length of histroy data:",len(history_data))
    
    
    n_node = len(greatcir_dis)
    knn_logi_mat = np.ones((n_node,n_node),dtype=float)
    # specify the position of known nodes inquired from the google distance matrix
    for i in range(n_node):
        for j in range(n_node):
            if logi_mat[i][j] == 1:
                knn_logi_mat[i][j] = circuity_factor[i][j]
    
    # infer the elements which are unknown in the smart distance matrix
    for i in range(n_node):
        for j in range(n_node):
            if logi_mat[i][j] == 0 and i!=j:
                alter_link = dict()
                
                for ii in range(n_node):
                    for jj in range(n_node):
                        if logi_mat[ii][jj]:
                            # distance between origins
                            dist_org = greatcir_dis[i,ii]
                            # distance between destinations
                            dist_dest = greatcir_dis[jj,j]
                            # save the sum of two distances
                            total_gc_dist = dist_org + dist_dest
                            alter_link[total_gc_dist] = circuity_factor[ii][jj]
                
                # calculate the distance using the historical data
                
                
                for idx in range(len(history_data)):
                    Coordinates1 = [location[i,0],location[i,1]]
                    Coordinates2 = [history_data[idx,0],history_data[idx,1]]
                    dist_org = gcdist(Coordinates1,Coordinates2)
                    print("dist_org:", dist_org)
                    
                    Coordinates1 = [location[j,0],location[j,1]]
                    Coordinates2 = [history_data[idx,2],history_data[idx,3]]
                    dist_dest = gcdist(Coordinates1,Coordinates2)
                    
                    # save the sum of two distances
                    total_gc_dist = dist_org + dist_dest
                    alter_link[total_gc_dist] = history_data[idx,4]
                
                # sort_dict's datatype is list
                sorted_dict = sorted(alter_link.items(), key=operator.itemgetter(0))
                
                logical_mean = 0
                if len(sorted_dict) >= num_adj:
                    
                    for item in range(num_adj):
#                         close_org = sorted_dict[item][0][0]
#                         close_dest = sorted_dict[item][0][1]
                        logical_mean = logical_mean + sorted_dict[item][1]
                        #logical_mean = logical_mean + circuity_factor[close_org][close_dest]
                    logical_mean = logical_mean/num_adj
                
                if logical_mean > 0:
                    knn_logi_mat[i][j] = logical_mean*sf
                              
    return knn_logi_mat*greatcir_dis

In [110]:
def matrix_completion(file_name,k_nn, scale_factor):
    
    # parameter setting
    inquiry_budget = 0.2
    print(file_name)
    
#     k_nn = 4
        
#     scale_factor = 1.1
    
    query_data = []
    
    path = r"C:\WYH\research project\MC Research-travel distance-p2-BASIC-INVERSE-GAUSSIAN\101TestData"
    
    file_path = os.path.join(path, file_name)
    print(file_path)
    
    File_Name
    
    # load the data 
    [google_distance,great_distance,location] = load_data(file_path)
    num_node = len(google_distance)
    # calculate the circuity factors
    CircuityFactor = google_distance/great_distance
    for i in range(num_node):
        for j in range(num_node):
            if np.isnan(CircuityFactor[i][j]):
                CircuityFactor[i][j] = 1
    
    # MaxElements denotes the total maximun nodes we can inquiry.
    MaxElements = int(inquiry_budget*num_node*num_node);
                
    # get the matrix showing which elements can be available
    access_matrix = element_inquiry(great_distance, MaxElements)
    
    # write the info of quried node pairs to local file
#     count_quried = 0
#     for i in range(num_node):
#         for j in range(num_node):
#             if access_matrix[i][j] != 0:
#                 count_quried = count_quried + 1
#                 node_info = [location[i][0],location[i][1],location[j][0],location[j][1],CircuityFactor[i][j]]
#                 query_data.append(node_info)
#     print("data_count:", count_quried)
#     print('length:',len(query_data))
    
#     np.savetxt(file_name.split('.')[0]+'_query_data'+'.txt', query_data) 
    
    # use knn framework to infer the whole distance matrix
    smart_distance = knn_matrix_completion(access_matrix,great_distance,CircuityFactor,k_nn,scale_factor,location, File_ID);
    
    whole_data = np.vstack((great_distance,smart_distance,google_distance,location))
    
    np.savetxt(file_name.split('.')[0]+'.txt', whole_data)
    
    
    

In [126]:
start = time.time()
path = r"C:\WYH\research project\MC Research-travel distance-p2-knn\101TestData"

# # parameter setting
# inquiry_budget = 0.2
 = 4
scale_factor = 1.1


for file in os.listdir(path):
    # run mainfunction
    matrix_completion(file)
    


end = time.time()
running_time = end-start
print(running_time)

C:\WYH\research project\MC Research-travel distance-p2-knn\101TestData\101x101_instance10.mat


c:\python372\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


the length of histroy data: 18369
1294.099901676178
